In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import SVHN
from sklearn.metrics import precision_score, recall_score, f1_score
import csv

In [2]:
# Step 1: Load the SVHN dataset
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomRotation(10),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = SVHN(root='./data', split='train', transform=train_transform, download=True)
test_dataset = SVHN(root='./data', split='test', transform=test_transform, download=True)

# Use a subset of the dataset (25%)
train_subset = Subset(train_dataset, range(0, len(train_dataset), 4))

100%|██████████| 182040794/182040794 [00:21<00:00, 8479151.19it/s] 


100%|██████████| 64275384/64275384 [00:06<00:00, 10187704.88it/s]


In [3]:
# Step 2: Preprocess the dataset
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
# Step 3: Choose pretrained models
# Implement LeNet-5 manually
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 16*5*5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

pretrained_models = {
    'LeNet-5': LeNet5(),
    'VGG-16': models.vgg16(pretrained=True),
    'ResNet-18': models.resnet18(pretrained=True),
    'ResNet-50': models.resnet50(pretrained=True),
    'ResNet-101': models.resnet101(pretrained=True)
}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 82.5MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future.

In [5]:
# Step 4: Load the pretrained weights for the chosen model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 5: Fine-tune the model on the SVHN dataset
num_epochs = 10
learning_rate = 0.001

# Initialize CSV writer
with open('model_metrics.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Model', 'Test Accuracy', 'Precision', 'Recall', 'F1-score'])

    for model_name, model in pretrained_models.items():
        model = model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()

        # Training loop with data augmentation and adjusted hyperparameters
        model.train()  # Set model to training mode

        for epoch in range(num_epochs):
            running_loss = 0.0
            correct = 0
            total = 0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                # Track the loss and accuracy
                running_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            # Print statistics every epoch
            epoch_loss = running_loss / len(train_loader.dataset)
            epoch_acc = correct / total * 100

            print(f"Model: {model_name}, Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

        print("Training finished for", model_name)

        # Evaluate the model on the test set
        model.eval()  # Set model to evaluation mode
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                predicted_labels.extend(predicted.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        test_accuracy = correct / total * 100
        print(f"Test Accuracy for {model_name}: {test_accuracy:.2f}%")

        # Calculate precision, recall, and F1-score
        precision = precision_score(true_labels, predicted_labels, average='macro')
        recall = recall_score(true_labels, predicted_labels, average='macro')
        f1 = f1_score(true_labels, predicted_labels, average='macro')

        # Write to CSV
        writer.writerow([model_name, test_accuracy, precision, recall, f1])

        # Performance report
        print(f"Performance Report for {model_name}:")
        print("--------------------------------------------------")
        print(f"Test Accuracy: {test_accuracy:.2f}%")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        print("--------------------------------------------------")
        print()

print("Output saved to model_metrics.csv")


Model: LeNet-5, Epoch [1/10], Loss: 2.1063, Accuracy: 24.19%
Model: LeNet-5, Epoch [2/10], Loss: 1.7406, Accuracy: 38.49%
Model: LeNet-5, Epoch [3/10], Loss: 1.5571, Accuracy: 46.01%
Model: LeNet-5, Epoch [4/10], Loss: 1.4133, Accuracy: 51.76%
Model: LeNet-5, Epoch [5/10], Loss: 1.3055, Accuracy: 55.65%
Model: LeNet-5, Epoch [6/10], Loss: 1.2208, Accuracy: 58.09%
Model: LeNet-5, Epoch [7/10], Loss: 1.1774, Accuracy: 60.22%
Model: LeNet-5, Epoch [8/10], Loss: 1.1199, Accuracy: 62.22%
Model: LeNet-5, Epoch [9/10], Loss: 1.0656, Accuracy: 64.28%
Model: LeNet-5, Epoch [10/10], Loss: 1.0346, Accuracy: 65.32%
Training finished for LeNet-5
Test Accuracy for LeNet-5: 73.89%
Performance Report for LeNet-5:
--------------------------------------------------
Test Accuracy: 73.89%
Precision: 0.7313
Recall: 0.7044
F1-score: 0.7081
--------------------------------------------------

Model: VGG-16, Epoch [1/10], Loss: 2.6061, Accuracy: 16.25%
Model: VGG-16, Epoch [2/10], Loss: 2.2535, Accuracy: 18.01

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: ResNet-18, Epoch [1/10], Loss: 2.0790, Accuracy: 32.37%
Model: ResNet-18, Epoch [2/10], Loss: 1.0332, Accuracy: 65.21%
Model: ResNet-18, Epoch [3/10], Loss: 0.7692, Accuracy: 75.12%
Model: ResNet-18, Epoch [4/10], Loss: 0.6758, Accuracy: 78.43%
Model: ResNet-18, Epoch [5/10], Loss: 0.6130, Accuracy: 80.96%
Model: ResNet-18, Epoch [6/10], Loss: 0.5728, Accuracy: 81.93%
Model: ResNet-18, Epoch [7/10], Loss: 0.6046, Accuracy: 81.11%
Model: ResNet-18, Epoch [8/10], Loss: 0.5150, Accuracy: 84.12%
Model: ResNet-18, Epoch [9/10], Loss: 0.4902, Accuracy: 84.48%
Model: ResNet-18, Epoch [10/10], Loss: 0.5131, Accuracy: 84.12%
Training finished for ResNet-18
Test Accuracy for ResNet-18: 89.33%
Performance Report for ResNet-18:
--------------------------------------------------
Test Accuracy: 89.33%
Precision: 0.8877
Recall: 0.8837
F1-score: 0.8844
--------------------------------------------------

Model: ResNet-50, Epoch [1/10], Loss: 1.5572, Accuracy: 51.28%
Model: ResNet-50, Epoch [2/10